In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import urllib


In [2]:
# URL of the Titanic dataset
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"

# Load the dataset into a DataFrame
titanic_data = pd.read_csv(url)

# View the first few rows of the dataset
print("First few rows of the dataset:")
print(titanic_data.head())


First few rows of the dataset:
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            3734

In [3]:
# Preprocess the data
# Drop irrelevant columns or handle missing values
titanic_data.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis=1, inplace=True)
titanic_data["Age"].fillna(titanic_data["Age"].median(), inplace=True)
titanic_data["Embarked"].fillna(titanic_data["Embarked"].mode()[0], inplace=True)

# Encode categorical variables
titanic_data = pd.get_dummies(titanic_data, columns=["Sex", "Embarked"], drop_first=True)


In [4]:
# Split the data into features (X) and target variable (y)
X = titanic_data.drop("Survived", axis=1)
y = titanic_data["Survived"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [5]:
# Build the TensorFlow model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Make predictions
y_pred = model.predict(X_test_scaled)
y_pred = (y_pred > 0.5).astype(int)


Epoch 1/50
18/18 [==============================] - 2s 20ms/step - loss: 0.6198 - accuracy: 0.6801 - val_loss: 0.5178 - val_accuracy: 0.7762
Epoch 2/50
18/18 [==============================] - 0s 11ms/step - loss: 0.5184 - accuracy: 0.7733 - val_loss: 0.4479 - val_accuracy: 0.7902
Epoch 3/50
18/18 [==============================] - 0s 8ms/step - loss: 0.4647 - accuracy: 0.7909 - val_loss: 0.4270 - val_accuracy: 0.8112
Epoch 4/50
18/18 [==============================] - 0s 10ms/step - loss: 0.4568 - accuracy: 0.8032 - val_loss: 0.4114 - val_accuracy: 0.8042
Epoch 5/50
18/18 [==============================] - 0s 8ms/step - loss: 0.4425 - accuracy: 0.8120 - val_loss: 0.4045 - val_accuracy: 0.8182
Epoch 6/50
18/18 [==============================] - 0s 9ms/step - loss: 0.4364 - accuracy: 0.8032 - val_loss: 0.4026 - val_accuracy: 0.8182
Epoch 7/50
18/18 [==============================] - 0s 8ms/step - loss: 0.4334 - accuracy: 0.8137 - val_loss: 0.4041 - val_accuracy: 0.8252
Epoch 8/50
18/18 

In [7]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"\n Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")


 Accuracy: 0.83
Precision: 0.84
Recall: 0.73
F1-score: 0.78
